## **The modified source code compatible with colab (gdown does not work for downloading):**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **LQVG weights trained on DIOR-RSVG and RSVG-HR**

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1uC9TAPOwiIbHcee6hSO_3b2Mwr-zDGtg?usp=drive_link -O drive/MyDrive/weights

In [ ]:
cd /content/drive/.shortcut-targets-by-id/18OjYDfO70rO2e-oLMPmNbu2Z1WH7nppd/LQVG


In [ ]:
!pip install torch torchvision torchaudio


In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install 'git+https://github.com/facebookresearch/fvcore'


In [ ]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'


In [ ]:
cd /content/drive/.shortcut-targets-by-id/18OjYDfO70rO2e-oLMPmNbu2Z1WH7nppd/LQVG/models/ops

In [ ]:
!python setup.py build install

In [ ]:
cd ../..

## Inference:

In [ ]:
!python3 inference_rsvg.py --dataset_file rsvg_mm --num_queries 10 --with_box_refine --binary --freeze_text_encoder \
--resume /content/drive/MyDrive/weights/RSVG-HR/checkpoint.pth --backbone resnet50

## **Visualize some results:**

In [ ]:
import os
from pathlib import Path
import math
from PIL import Image
import matplotlib.pyplot as plt

# --- Config ---
IDS = [120, 159, 316, 328, 377, 502, 507, 510]
folder = "/content/drive/MyDrive/LQVG/test_output/test"  # Input folder
save_path = "/content/drive/MyDrive/grid/grid.png"       # Output file
cols = 4                                                 # Grid columns

# --- Ensure save directory exists ---
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# --- Functions ---
def load_images(root, ids):
    """Load images by IDs from a folder."""
    imgs = []
    for i in ids:
        name = f"UASs_{i}.jpg"
        path = Path(root) / name
        if not path.is_file():
            print(f"❌ Missing: {path}")
            continue
        imgs.append(Image.open(path).convert("RGB"))
        print(f"✅ Loaded: {path}")
    return imgs

def show_grid(images, cols=4, save=None):
    """Display images in a grid and optionally save."""
    if not images:
        print("⚠ No images to display.")
        return

    rows = math.ceil(len(images) / cols)

    # Scale figure size based on image dimensions
    img_width, img_height = images[0].size
    fig_width = cols * img_width / 100
    fig_height = rows * img_height / 100

    plt.figure(figsize=(fig_width, fig_height))
    for idx, img in enumerate(images, 1):
        ax = plt.subplot(rows, cols, idx)
        ax.imshow(img)
        ax.axis("off")

    if save:
        plt.savefig(save, dpi=150, bbox_inches='tight', pad_inches=0)
        print(f"💾 Saved grid to: {save}")

    plt.show()

# --- Main ---
imgs = load_images(folder, IDS)
show_grid(imgs, cols=cols, save=save_path)


## Training:

In [ ]:
!wget -O datasets/coco_eval.py https://raw.githubusercontent.com/facebookresearch/detr/main/datasets/coco_eval.py

In [ ]:
!python main.py \
    --dataset_file rsvg_mm \
    --rsvg_mm_path "/content/drive/MyDrive/data" \
    --binary \
    --with_box_refine \
    --batch_size 1 \
    --num_frames 1 \
    --epochs 70 \
    --lr_drop 40 \
    --num_queries 10 \
    --output_dir "/content/drive/MyDrive/rsvg_dirs/r50_bidrection_fusion_10query_70epo_multiscale" \
    --backbone resnet50 \
    --device cuda

In [ ]:
jupyter nbconvert --to pdf LQVG-test.ipynb
